<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b>

Приветствую тебя еще раз!) Спасибо за то, что доработал проект!

Мои комментарии будут также обозначены разными цветами, но с пометкой 2:
    
    «КОММЕНТАРИЙ РЕВЬЮВЕРА 2: ... »

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 
    
    
Роман, ты доработал проект, желаю успехов в дальнейшем обучении!

</div>

<b>КОММЕНТАРИЙ РЕВЬЮВЕРА</b>

Привет, Роман👋!)

Меня зовут Обидин Андрей, я осуществлял проверку проекта. Может перейдем сразу на <b>ты</b>? Если это неприемлемо, то обязательно дай знать)

Благодарю тебя за то, что вовремя всё сдал и хорошо поработал!)

`Роман, ты хорошо выполнил проект, у меня по сути нет к тебе замечаний, но есть советы и рекомендации на будущее. Посмотри проект, может быть у тебя возникнут вопросы. В конце проекта я дал развернутую рекомендацию по тому, как можно еще улучшить модели, обо всем этом ты узнаешь более подробно чуть позже, так что не гони коней как говорится) Мне просто хотелось, чтобы у тебя в голове уже сформировалась определенная стратегия по решению проблем с которыми мы столкнулись в данном кейсе)`

Далее по ходу работы ты сможешь встретить мои комментарии, они будут отмечены разными цветами:

<div class="alert alert-block alert-success">
Похвала и возможности развития
</div>
<div class="alert alert-block alert-warning">
Некритичные замечания, но стоит обратить внимание
</div>
<div class="alert alert-block alert-danger">
Грубая ошибка, необходимо исправить
</div>


Заранее прошу не удалять мои комментарии, это важно. Если у тебя возникнут вопросы, обозначь их, я с радостью тебе помогу!)

Как вариант, можешь обозначить их так:
<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА:</b>
</div>

# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».


В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Построим модель с максимально большим значением *accuracy*. Цель - 0.75. Проверим *accuracy* на тестовой выборке.

## Откроем и изучим файл

Импортируем необходимые библиотеки

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Метрика MSE не нужна, так как мы решаем задачу классификации, а mean_squared_error это метрика регрессии

</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА:</b>
    Да спасибо, забыл удалить, скопипастил)
</div>

Откроем файл с данными

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Предобработка данных не требуется, пропущенных значений нет.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
В задачах классификации полезно смотреть на количество объектов того или иного класса, чтобы понять есть ли дисбаланс классов в данных или нет

</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА:</b>
    Добавил
</div>

Посмотрим на соотношение между классами

In [9]:
df.is_ultra.value_counts()/df.shape[0]

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 
    
    
Наблюдается явный дисбаланс классов, это может негативно сказаться на работе модели, так как она будет настраиваться на паттерны доминирующего класса

</div>

## Разобьем данные на выборки

Извлекем признаки для обучения:

In [10]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Выделим 25% данных для тестовой выборки, остальные — для валидационной.

In [11]:
features_train, features_test_valid, target_train, target_test_valid = (train_test_split(features,
                                                                               target, 
                                                                               test_size=0.40, 
                                                                               random_state=12345) 
                                                             )

In [12]:
features_test, features_valid, target_test, target_valid = (train_test_split(features_test_valid,
                                                                               target_test_valid, 
                                                                               test_size=0.50, 
                                                                               random_state=12345) 
                                                             )

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Хорошо

</div>

## Исследуем модели

### Модель дерево решений

Проверим точность модели на валидационной выборке и найдем оптимальные гиперпараметры

In [47]:
%%time
1 + 1
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)# инициализируйте модель DecisionTreeRegressor с параметром random_state=12345 и max_depth=depth
    model.fit(features_train, target_train) # обучите модель на тренировочной выборке
    predictions_valid = model.predict(features_valid) # получите предсказания модели на валидационной выборке
    #result = mean_squared_error(target_valid, predictions_valid) ** (1/2)# посчитайте значение метрики rmse на валидационной выборке
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print("Точность наилучшей модели на валидационной выборке:", best_result, "Глубина дерева:", best_depth)

Точность наилучшей модели на валидационной выборке: 0.7993779160186625 Глубина дерева: 7
CPU times: user 73.8 ms, sys: 32 µs, total: 73.8 ms
Wall time: 71.5 ms


<div class="alert alert-block alert-warning">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    

Ты измеряешь не точность, а долю правильных ответов. Точность это немного другая метрика классификации, с которой ты познакомишься чуть позже, поэтому не путай терминологию, а то потом сложновато будет)
    
P.s. это замечание характерно для всех моделей
</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА:</b>
    Точно, это же так называемая "правильность". Спасибо!
</div>

### Модель случайный лес

In [13]:
%%time
1 + 1
best_model = None
best_result = 0
best_depth = 0
for est in range(1, 11):
    for depth in range(1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print("Точность наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est,  "Глубина дерева:", best_depth)

Точность наилучшей модели на валидационной выборке: 0.8133748055987559 Количество деревьев: 8 Глубина дерева: 9
CPU times: user 2.24 s, sys: 0 ns, total: 2.24 s
Wall time: 2.24 s


<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Можно было бы сделать второй вложенный цикл и прогнать глубину

</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА:</b>
    Добавил, результат получился лучше
</div>

### Модель логистическая регрессия

In [53]:
%%time
1 + 1
best_model = None
best_result = 0
best_depth = 0
model = LogisticRegression(random_state=12345) 
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print("Точность наилучшей модели на валидационной выборке:", result)

Точность наилучшей модели на валидационной выборке: 0.7402799377916018
CPU times: user 21.9 ms, sys: 34 µs, total: 21.9 ms
Wall time: 20.1 ms


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Вывод:** Видим что у случайного леса и решающего дерева одинаково высокое качество, у дерева решений быстродействие выше. Логистическая регрессия имеет наиболее высокое быстродействие и низкую точность.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Молодец, хорошо провел построение и анализ моделей

</div>

## Проверим модель на тестовой выборке

Проверим модель показавшую лучший результат на валидационной выборке - дерево решений на тестовой выборке

In [64]:
%%time
1 + 1
model = DecisionTreeClassifier(random_state=12345, max_depth=7)
model.fit(features_train, target_train) 
predictions_test = model.predict(features_test) 
result = accuracy_score(target_test, predictions_test)
print("Точность модели на тестовой выборке:", result)

Точность модели на тестовой выборке: 0.7822706065318819
CPU times: user 13.1 ms, sys: 21 µs, total: 13.1 ms
Wall time: 11 ms


Результат качества выше целевого 0,75.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Да, дерево действительно дает метрику лучше 0.75 на тестовых данных

</div>

## Проверим модели на адекватность

Чтобы оценить адекватность модели в задачах классификации, нужно сравнить её со случайной. Ответим на все объекты одним и тем же классом. Чтобы оно не сильно расходилось с правдой, примем за этот класс наиболее популярный тариф.

Проверим соотношение количества клиентов с тарифом Ультра и Смарт

In [68]:
df.is_ultra.mean()

0.30647168637212197

30% клиентов пользуются тарифом ультра. Остальные тарифом смарт.

In [87]:
predictions_mean = pd.Series(0, index=target.index)

In [88]:
result = accuracy_score(target, predictions_mean)
print("Точность модели при предсказании одного и того же тарифа:", result)

Точность модели при предсказании одного и того же тарифа: 0.693528313627878


**Вывод:** точность полученная с использованием модели выше, чем при случайных предсказанных значениях. Целесообранее использовать модель для предложения пользователям нового тарифа.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
В качестве дополнительной проверки на адекватность можно было подсчитать AUC_score модели и построить ROC-кривую. Об этих метриках ты узнаешь чуть позже. Дело в том, что auc случайной модели, которая 50\50 прогнозирует класс = 0.5, если auc твоей модели будет выше данного показателя, то очевидно модель твоя работает уже лучше случайной)
    
    
Как еще можно было улучшить модели:
    
    
* в данных есть явный дисбаланс классов, ты уже это заметил, как правило это приводит к настраиванию моделей на доминирующий класс. У нас может быть высокая доля правильных ответов, но по факту модель может иметь уже не такую хорошую точность \ полноту или ф-меру
    
* для решения проблемы дисбаланса и поднятия метрик используют следующие методы: upsampling (увеличение объектов меньшего класса), downsampling (уменьшение объектов доминирующего класса), balanced (взвешивание классов)
    
* также можно было провести масштабирование признаков
    
    
P.s. более подробно об этих методах ты узнаешь чуть позже, просто мне хотелось, чтобы ты уже сейчас примерно понимал, как можно двигаться и улучшать модели дальше)

    
`Роман, ты хорошо выполнил проект, молодец! Как таковых замечаний у меня нет, больше рекомендаций, изучи их пожалуйста)`
</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА:</b>
    Спасибо! Если честно не понял ничего, дисбаланс классов то что 30 к 70? При этом когда делим на тестовые и валидационные выборки у нас ведь в этом же соотношении по умолчанию классы делятся?
</div>

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 
    
    
Да, дисбаланс это то, что ты в начале проекта посмотрел

Чтобы учесть классы при разбиении можно установить параметр стратификации, но вообще проще потом при обучении моделей решить проблему дисбаланса путем взвешивания классов, upsampling \ downsampling

Переходи к следующему курсу, там все это изложено более подробно)

</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
